# Quantum Computing

Quantum Computing uses physical, quantum mechanical processes to gain
a computing advantage over classical computing.  For some problems, it
is believed a reliable quantum computer will out-perform a classical
computer.  This includes fundamental quantum
calculations in material sciences and quantum chemistry, as well as
some mathematical operations, such as factoring numbers composed from
very large primes.

Two processes fundamental to quantum computing are:

Superposition: Qubits (quantum bits) can be placed into a state that is
    between $0$ and $1$ called superposition.
    
Entanglement: Qubits can be mathematically correlated, such that knowledge of one of the qubits reveals
knowledge of the other qubits.

RPI has an IBM Quantum One computer with and Eagle chip, which has 127 qubits.   The Quantum System One is programmed
using quantum gates, via the Qiskit API, using Python.  IBM is working on a C API, and much of Qiskit is written in Rust.


# A "Simple" Quantum Program

This program will create what is called a Bell state from two qubits, and submit it to a quantum computer.

A Bell state demonstrates entanglement between two qubits.  A quantum circuit with two qubits is created. One
qubit is placed into superposition, and then used as the "control qubit" for a second qubit, using a CNOT gate.

In [ ]:
from qiskit_ibm_runtime import QiskitRuntimeService
from qiskit import QuantumCircuit
from qiskit.quantum_info import SparsePauliOp
from qiskit.transpiler.preset_passmanagers import generate_preset_pass_manager
from qiskit_ibm_runtime import EstimatorV2 as Estimator
from qiskit_ibm_runtime import SamplerV2 as Sampler

# You'll need to specify the credentials when initializing QiskitRuntimeService.
# The backend is needed for the later transpile step.
service = QiskitRuntimeService()
backend = service.backend("ibm_rensselaer")

# Create a bell state
qc = QuantumCircuit(2)
qc.h(0)
qc.cx(0,1)
qc.measure_all()

# Draw circuit.
qc.draw("mpl")


# Tanspiling

Before a circuit can be run, the abstract circuit must be transformed into an equivelent circuit written in the
quantum computers native Instruction Set Architecture (ISA).  This process is called "transpiling."

In [ ]:
pm = generate_preset_pass_manager(backend=backend, optimization_level=1)
isa_circuit = pm.run(qc)

# Draw the transpiled circuit
isa_circuit.draw("mpl", idle_wires=False)


Notice how different gates are being used.  While they are different, and may be more difficult for you to read at this
time, both circuits do the same thing.

Now send the circuit to a quantum computer.  Note, the computer, and queue priority, are determined by the QiskitRuntimeService
you specified in the "Runtime" program.

In [ ]:
# Run the transpiled circuit 
sampler = Sampler(backend)
job = sampler.run([isa_circuit])
print(f"job id: {job.job_id()}")

result = job.result()
print(result)



# Visualize the Output

The circuit was run 1024 times, and the results of each run measured.  The "raw" output above can be used visualized
as a histogram, or used as the input to any classical or quantum program.

In [ ]:
# Get results from Databin
pub_result = result[0]
counts = pub_result.data.meas.get_counts()
print(f"Counts for the meas output register: {counts}")

# Plot results
from qiskit.visualization import plot_histogram
plot_histogram(counts)


# Unentangled Qubits

Lets try a program that is almost like the previous program.  The differences are: Both qubits are separately placed in superposition,
and they are not entangled.

In [ ]:
# Put non-entabled qubits into superposition
qc2 = QuantumCircuit(2)
qc2.h(0)
qc2.h(1)
qc2.measure_all()

# Draw circuit.
qc2.draw("mpl")


Run the program, and plot the results on a histogram.

In [ ]:
pm = generate_preset_pass_manager(backend=backend, optimization_level=1)
isa_circuit = pm.run(qc2)
sampler = Sampler(backend)
job = sampler.run([isa_circuit])
print(f"job id: {job.job_id()}")

result = job.result()

pub_result = result[0]
counts = pub_result.data.meas.get_counts()
print(f"Counts for the meas output register: {counts}")

# Plot results
from qiskit.visualization import plot_histogram
plot_histogram(counts)


# Exercise 1

Compare the results of the two quantum programs.  Why they are different?
